In [1]:
df = spark.sql("SELECT * FROM Bing_news.latest_new_tbl")
display(df)

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2b7aa498-a0be-4232-9380-fb598fe806ed)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 4, Finished, Available, Finished)

In [3]:
model=(AnalyzeText()
       .setTextCol("snippet")
       .setKind("SentimentAnalysis")
       .setOutputCol("response")
       .setErrorCol("error")
)
result=model.transform(df)


StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 5, Finished, Available, Finished)

In [4]:
display(result)

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b2295ed1-ddc7-40b6-9111-f0d714dd0ebb)

In [5]:
from pyspark.sql.functions import col
sentiment_df=result.withColumn("sentiment",col("response.documents.sentiment"))
display(sentiment_df)

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f41c742-a62a-4560-93fc-76772b41efea)

In [6]:
sentiment_final_df=sentiment_df.drop("error","response")
display(sentiment_final_df)

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, af6a7d82-1b47-4089-a075-b7a553951b8c)

In [7]:
from pyspark.sql.utils import AnalysisException
 
try:
    table_name='Bing_news.latest_sentiment_analysis'
    sentiment_final_df.write.format("delta").saveAsTable(table_name)
except:
    print("Table Already Exists!")
    sentiment_final_df.createOrReplaceTempView("vw_sentiment_final_df")
 
    spark.sql(f"""
    MERGE INTO Bing_news.latest_sentiment_analysis AS target_table
    USING vw_sentiment_final_df AS source_view
    ON source_view.link = target_table.link
    WHEN MATCHED AND (
    source_view.source <> target_table.source OR
    source_view.title <> target_table.title OR
    source_view.snippet <> target_table.snippet OR
    source_view.link <> target_table.link OR
    source_view.date = target_table.date OR
    source_view.day <> target_table.day OR
    source_view.month <> target_table.month OR
    source_view.year <> target_table.year OR
    source_view.hour <> target_table.hour OR
    source_view.weekday <> target_table.weekday
    )
    THEN UPDATE SET *
   
    WHEN NOT MATCHED THEN INSERT *
    """)
 

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 9, Finished, Available, Finished)

Table Already Exists!


In [8]:
%%sql
 
SELECT * from Bing_news.latest_sentiment_analysis

StatementMeta(, b6415f45-7077-4607-a047-9a0356334f3d, 10, Finished, Available, Finished)

<Spark SQL result set with 78 rows and 11 fields>